In [3]:
import pandas as pd
import os
import glob
import numpy as np
from sqlalchemy import create_engine

In [4]:
def extract_data(data_dir:str) -> pd.DataFrame:
    # extract and combine all csv files.
    all_files = glob.glob(os.path.join(data_dir , "*.csv"))
    li = []
    for filename in all_files:
        df = pd.read_csv(filename)
        li.append(df)
    frame = pd.concat(li, axis=0, ignore_index=True)
    return frame

In [16]:
data_directory = r'.\data\products'
# E

extracted_data = extract_data(data_directory)
extracted_data.shape

(1500, 35)

In [17]:
sellers = pd.read_csv('data/sellers.csv')
sellers.shape

(460, 8)

In [21]:
print(extracted_data.columns)
extracted_data[['owner','owner_link']]

Index(['link', 'id', 'title', 'images', 'member_since', 'description', 'owner',
       'reviews', 'google_maps_locatoin_link', 'long', 'lat', 'owner_link',
       'price', 'City', 'Neighborhood', 'Zoned for', 'Land Area', 'Facade',
       'Property Mortgaged?', 'Lister Type', 'Payment Method', 'Category',
       'Subcategory', 'Nearby', 'Bedrooms', 'Bathrooms', 'Furnished?',
       'Surface Area', 'Floor', 'Building Age', 'Main Amenities',
       'Reference ID', 'Number of Floors', 'Additional Amenities',
       'Property Status'],
      dtype='object')


,owner,owner_link
0,مكتب العال و الروضه العقاري,https://opensooq.com/en/mid/member-5443819
1,الكسواني للاسكان,https://opensooq.com/en/mid/member-1534066827100
2,شركه رائد العساف وشريكه,https://opensooq.com/en/mid/member-145796546167
3,بلال نجم,https://opensooq.com/en/mid/member-163196666279
4,الكسواني للاسكان,https://opensooq.com/en/mid/member-1534066827100
...,...,...
1495,F J T,https://opensooq.com/en/mid/member-3782353
1496,جيفارا العقاري,https://opensooq.com/en/shops/جيفارا-العقاري-7...
1497,ابو الخير,https://opensooq.com/en/mid/member-145700906456
1498,الثقه العقاريه,https://opensooq.com/en/shops/الثقه-العقاريه-1...


In [26]:
extracted_data['owner'].isnull().sum()

22

In [27]:
all = extracted_data.merge(sellers, on='owner_link', how='left')

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import re

In [10]:
print(requests.get('https://opensooq.com'))

<Response [200]>


In [8]:
links = pd.read_csv('data/links.csv')
links = links.iloc[5000:]

In [9]:
len(links)

27850

In [18]:
for row in range(5000,len(links)):
    link = links.loc[row,'link']   

In [ ]:
def scrape_links(URL:str) -> str:
    df = []
    i = 1
    while True:
        url = f'{URL}&page={i}'
        r = requests.get(url) 
        if r.title.text == '404 Not Found':
            break
        else:
            soup = BeautifulSoup(r.content)
            table = soup.find('section', attrs = {'id':'serpMainContent'})
            for row in table.findAll('div', {'class':re.compile('sc-21acf5d5-0 jhHVZS mb-32 relative radius-8 grayHoverBg whiteBg boxShadow2')}): 
                data = {}
                data['id'] = row.a['href'][11:20]
                data['link'] = 'https://opensooq.com'+row.a['href']
                data['price'] = row.find('div',{'class':'priceColor bold alignSelfCenter font-18 ms-auto'}).text
                df.append(data)
            if i%50==0:
                time.sleep(60)
                print(f'Pages Scrpaed: {i}')
            url = URL
            i+=1
    pd.DataFrame(df).to_csv('data/links.csv',index_label=False)
    return "Scraping Products Links is Done!"